# New Section

#Practical no 1

In [ ]:
import tensorflow as tf
import numpy as np

c = tf.constant(np.array([[7,2],[3,4]]))
print(c)

print(c[0])
print(c[1,0])

print(tf.reshape(c,[1,4]))

f = c+ c
print(f)

g = tf.matmul(c,c)
print('a @ a:\n',g)


x = tf.constant(3.0)
y = x * 2 + 1  # Immediate evaluation
print("Eager: x*2 + 1 =", y)
print(x)

# Practical no 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data_group1 = np.random.normal(loc = 10 , scale = 10 , size = 20)
data_group2 = np.random.normal(loc = 100 , scale = 10 , size = 20)

plt.hist(data_group1,label="Group1", color = "blue")
plt.hist(data_group2,label="Group2", color = "red")
plt.show()

#practical no 4

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = [
    "artificial intelligence is cool",
    "machine learning is fun",
    "ai learning uses neural netowrks"
]

model = TfidfVectorizer()
x = model.fit_transform(sentences)

print("Vocabolary", model.get_feature_names_out())
print("Most similar to learining :", x.toarray()[:,model.vocabulary_["is"]])

#practical no 5+


In [ ]:
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt


(x,_),_=tf.keras.datasets.mnist.load_data()
x=x/255.; x=np.expand_dims(x,-1)


i=tf.keras.Input((28,28,1))
o=tf.keras.layers.Conv2D(32,3,padding="same",activation="relu")(i)
o=tf.keras.layers.Conv2D(1,3,padding="same")(o)
model=tf.keras.Model(i,o)


img=tf.random.normal((1,28,28,1))
for _ in range(80):
  img = img-model(img)*0.1+tf.random.normal(img.shape)*0.02

plt.imshow(tf.squeeze(tf.clip_by_value(img,0,1)),cmap="gray"); plt.axis("off")


# Practical no 6

In [ ]:

!pip install diffusers transformers accelerate torch

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cpu")   # Use CPU for generation

prompt = "A beautiful scenic sunset over mountains"

image = pipe(prompt).images[0]

image.show()
image.save("generated_image.png")



# Practical no 7

In [ ]:
!pip install transformers peft torch accelerate -q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import LoraConfig, get_peft_model, TaskType
import math


name = "distilgpt2"
tok = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name)


config = LoraConfig(r=4, lora_alpha=8, task_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, config)


text = "AI is changing the world."
inputs = tok(text, return_tensors="pt")
loss = model(**inputs, labels=inputs["input_ids"]).loss

print("Loss:", loss.item())
print("Perplexity:", math.exp(loss.item()))

gen = pipeline("text-generation", model=model, tokenizer=tok)
print(gen("AI will", max_length=40, do_sample=True)[0]["generated_text"])


#Practical no 8


In [ ]:

!pip install transformers datasets evaluate torch rouge_score -q

from transformers import pipeline
import evaluate


summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)


text = """Artificial Intelligence (AI) is transforming industries by automating tasks,
improving efficiency, and enabling new innovations such as self-driving cars and smart assistants."""


summary = summarizer(text, max_length=40, min_length=10, do_sample=False)[0]['summary_text']
print("Generated Summary:\n", summary)


rouge = evaluate.load("rouge")
results = rouge.compute(predictions=[summary], references=["AI is changing industries through automation and innovation."])
print("\nROUGE Scores:", results)

#practical no 9

In [ ]:
!pip install faiss-cpu sentence-transformers transformers torch -q

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss, numpy as np

docs = [
    "Taj Mahal is in Agra, India.",
    "Eiffel Tower is in Paris, France.",
    "Great Wall of China is very long.",
    "Mount Everest is the tallest mountain."
]

embed = SentenceTransformer('all-MiniLM-L6-v2')
vecs = embed.encode(docs)

index = faiss.IndexFlatL2(vecs.shape[1])
index.add(vecs)

q = "Where is the Eiffel Tower?"
q_vec = embed.encode([q])
_, idx = index.search(q_vec, 1)
ctx = docs[idx[0][0]]

gen = pipeline("text-generation", model="distilgpt2")
print(gen(f"Context: {ctx}\nQ: {q}\nA:", max_length=40)[0]["generated_text"])


Practical no 10

In [ ]:
!pip install transformers torch psutil -q

import torch, time, psutil
from transformers import AutoModelForSequenceClassification, AutoTokenizer

name = "distilbert-base-uncased"
tok = AutoTokenizer.from_pretrained(name)
m32 = AutoModelForSequenceClassification.from_pretrained(name)

txt = "AI is transforming the world."
x = tok(txt, return_tensors="pt")

t = time.time();
with torch.no_grad(): m32(**x)
print("FP32:", round(time.time() - t, 4), "s")
print("Mem before:", psutil.virtual_memory().percent, "%")

m8 = torch.quantization.quantize_dynamic(m32, {torch.nn.Linear}, dtype=torch.qint8)

t = time.time();
with torch.no_grad(): m8(**x)
print("INT8:", round(time.time() - t, 4), "s")
print("Mem after:", psutil.virtual_memory().percent, "%")


practical no 11


In [ ]:
!pip install transformers torch -q
from transformers import pipeline

gen = pipeline("text-generation", model="gpt2")
p = "A nurse is usually"
out = gen(p, max_length=20)[0]["generated_text"]

bad = ["he","she","his","her"]
clean = " ".join("[filtered]" if w.lower() in bad else w for w in out.split())

print("Original:\n", out)
print("\nMitigated:\n", clean)
print("\nBias before / after:",sum(w.lower() in bad for w in out.split()),"/", sum(w.lower() in bad for w in clean.split()))


Practical no 12


In [ ]:
import random

class Agent:
    def __init__(self, name): self.name, self.energy = name, 100
    def choose(self): return random.choice(["coop","comp"])
    def act(self, other):
        a, b = self.choose(), other.choose()
        if a=="coop" and b=="coop": self.energy+=10; other.energy+=10; r="🤝 both cooperated"
        elif a=="comp" and b=="coop": self.energy+=20; other.energy-=10; r=f"{self.name} exploited {other.name} 😈"
        elif a=="coop" and b=="comp": self.energy-=10; other.energy+=20; r=f"{other.name} exploited {self.name} 😢"
        else: self.energy-=5; other.energy-=5; r="⚔️ both competed"
        return r

A, B = Agent("A"), Agent("B")

for i in range(5):
    print(f"\nRound {i+1}")
    print(A.act(B))
    print(f"A:{A.energy} | B:{B.energy}")

print("\nWinner:", "A✅" if A.energy>B.energy else "B✅" if B.energy>A.energy else "Tie 🤝")

